1. Import Libraries

In [2]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("data.json").read()
intents = json.loads(data_file)

# 2. Data pre-processing

In [3]:
# preprocessing the json data
# tokenization
#nltk.download('punkt')
#nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### 3Tokenisasi


In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('texts.pkl','wb')) 
pickle.dump(classes,open('labels.pkl','wb'))

69 documents
68 classes ['breast_cancer1', 'breast_cancer2', 'breast_cancer3', 'endometriosis1', 'endometriosis2', 'endometriosis3', 'fertility1', 'fertility2', 'fertility3', 'fibroid1', 'fibroid2', 'fibroid3', 'greeting1', 'greeting2', 'greeting3', 'greeting_malam', 'greeting_sore', 'infeksi1', 'infeksi2', 'infeksi3', 'kanker_serviks1', 'kanker_serviks2', 'kanker_serviks3', 'kb1', 'kb2', 'kb3', 'menopause1', 'menopause2', 'menopause3', 'menstruasi1', 'menstruasi2', 'menstruasi3', 'noanswer', 'osteoporosis1', 'osteoporosis2', 'osteoporosis3', 'ovulasi1', 'ovulasi2', 'ovulasi3', 'payudara_bengkak1', 'payudara_bengkak2', 'payudara_bengkak3', 'pcos1', 'pcos2', 'pcos3', 'pengalaman1', 'pengalaman2', 'pengalaman3', 'penutup1', 'penutup2', 'perdarahan_postmenopause1', 'perdarahan_postmenopause2', 'perdarahan_postmenopause3', 'polip1', 'polip2', 'polip3', 'pregnancy1', 'pregnancy2', 'pregnancy3', 'sts1', 'sts2', 'sts3', 'tingkat_hormon1', 'tingkat_hormon2', 'tingkat_hormon3', 'vulvitis1', 'vu

# 4. Creating Training Data



In [5]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle features and converting it into numpy arrays
random.shuffle(training)

# Split training into two separate lists
train_x = [row[0] for row in training]
train_y = [row[1] for row in training]

# Convert the lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")

Training data created


# 5. Creating Modeling 



In [6]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('models.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0252 - loss: 4.2484      
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0243 - loss: 4.2393     
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0139 - loss: 4.1338     
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 4.1666 
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0333 - loss: 4.1283     
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0232 - loss: 4.1261     
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0986 - loss: 4.1160 
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1044 - loss: 3.9361     
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1330 - loss: 3.9407 
Epoch 10/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0909 - loss: 3.8969     
Epoch 11/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1726 - loss: 3.8198 
Epoch 12/200
14/14 



**************************************************

Model Created Successfully!
